In [1]:
import os
import geopandas as gpd
import xarray as xr
from shapely.geometry import Point
import pandas as pd
import p_auxiliary as aux
import p_location_class as plc

In [2]:
data_dir = os.path.join(os.getcwd(),'data')
land = xr.open_dataset(os.path.join(data_dir,'MCD12C1.A2022001.061.2023244164746.hdf'), engine="netcdf4")

In [4]:
land_df = land['Land_Cover_Type_1_Percent'].to_dataframe()
# Reset the index and drop the unnecessary index levels
land_df.reset_index(level=['YDim:MOD12C1', 'XDim:MOD12C1', 'Num_IGBP_Classes:MOD12C1'], inplace=True)

# Rename the 'Land_Cover_Type_1_Percent' column
land_df.rename(columns={'Land_Cover_Type_1_Percent': 'Percentage'}, inplace=True)
land_df

# Two additional exclusions are made to the total amount of land; 
# land which has been designated as protected in the ‘World Database on Protected Areas’33 
# and land which is steeply sloping (i.e. >15 degrees)

YDim:MOD12C1  XDim:MOD12C1  Num_IGBP_Classes:MOD12C1  Percentage
0                     0             0                         0       100.0
1                     0             0                         1         0.0
2                     0             0                         2         0.0
3                     0             0                         3         0.0
4                     0             0                         4         0.0
...                 ...           ...                       ...         ...
440639995          3599          7199                        12         0.0
440639996          3599          7199                        13         0.0
440639997          3599          7199                        14         0.0
440639998          3599          7199                        15         0.0
440639999          3599          7199                        16         0.0

[440640000 rows x 4 columns]

In [5]:
# Calculate latitude and longitude from grid indices
grid_size = 0.05  # 0.05 degrees
land_df['Latitude'] = 90.0 - grid_size * land_df['YDim:MOD12C1']
land_df['Longitude'] = -180.0 + grid_size * land_df['XDim:MOD12C1']
land_df

YDim:MOD12C1  XDim:MOD12C1  Num_IGBP_Classes:MOD12C1  Percentage  \
0                     0             0                         0       100.0   
1                     0             0                         1         0.0   
2                     0             0                         2         0.0   
3                     0             0                         3         0.0   
4                     0             0                         4         0.0   
...                 ...           ...                       ...         ...   
440639995          3599          7199                        12         0.0   
440639996          3599          7199                        13         0.0   
440639997          3599          7199                        14         0.0   
440639998          3599          7199                        15         0.0   
440639999          3599          7199                        16         0.0   

           Latitude  Longitude  
0             90.00    -180.00  
1             90.00    -180.00  
2             90.00    -180.00  
3             90.00    -180.00  
4             90.00    -180.00  
...             ...        ...  
440639995    -89.95     179.95  
440639996    -89.95     179.95  
440639997    -89.95     179.95  
440639998    -89.95     179.95  
440639999    -89.95     179.95  

[440640000 rows x 6 columns]

In [9]:
from geopy.distance import geodesic
import numpy as np

# Function to calculate the area of a grid cell using geopy
def calculate_grid_area_geopy(lat1, lon1, lat2, lon2):
    # Calculate the distance between two diagonal corners of the grid cell
    dist = geodesic((lat1, lon1), (lat2, lon2)).meters

    # Calculate the area of the grid cell as a square
    area = dist**2

    return area

# Calculate the area for each grid cell in the dataframe using np.vectorize
vectorized_area_calculation = np.vectorize(calculate_grid_area_geopy)
areas = vectorized_area_calculation(
    land_df['Latitude'][:-1].values, 
    land_df['Longitude'][:-1].values, 
    land_df['Latitude'][1:].values, 
    land_df['Longitude'][1:].values
)

# Set the 'Area' column in the dataframe
land_df['Area'][:-1] = areas

# Print the updated dataframe with the 'Area' column
print(land_df)


: 

In [6]:
ref_power_availability = {
    'Land Cover Type': [
        'Evergreen Needleleaf Forests',
        'Evergreen Broadleaf Forests',
        'Deciduous Needleleaf Forests',
        'Deciduous Broadleaf Forests',
        'Mixed Forests',
        'Closed Shrublands',
        'Open Shrublands',
        'Woody Savannas',
        'Savannas',
        'Grasslands',
        'Permanent Wetlands',
        'Croplands',
        'Urban and Built-up Lands',
        'Cropland, Natural Vegetation, Mosaic',
        'Permanent Snow and Ice',
        'Barren',
        'Water Bodies'
    ],
    'MODIS Number': [
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17
    ],
    'Wind': [
        '-', '-', '-', '-', '-', 0.5, 0.5, 0.2, 0.2, 0.2, '-', 0.05, '-', 0.05, '-', 1, '-'
    ],
    'Solar': [
        '-', '-', '-', '-', '-', 0.5, 0.5, 0.2, 0.2, 0.2, '-', '-', 0.03, '-', '-', 1, '-'
    ]
}

# Replace '-' with 0 in Wind and Solar columns
ref_power_availability['Wind'] = [0 if x == '-' else float(x) for x in ref_power_availability['Wind']]
ref_power_availability['Solar'] = [0 if x == '-' else float(x) for x in ref_power_availability['Solar']]

pd.DataFrame(ref_power_availability)

Land Cover Type  MODIS Number  Wind  Solar
0           Evergreen Needleleaf Forests             1  0.00   0.00
1            Evergreen Broadleaf Forests             2  0.00   0.00
2           Deciduous Needleleaf Forests             3  0.00   0.00
3            Deciduous Broadleaf Forests             4  0.00   0.00
4                          Mixed Forests             5  0.00   0.00
5                      Closed Shrublands             6  0.50   0.50
6                        Open Shrublands             7  0.50   0.50
7                         Woody Savannas             8  0.20   0.20
8                               Savannas             9  0.20   0.20
9                             Grasslands            10  0.20   0.20
10                    Permanent Wetlands            11  0.00   0.00
11                             Croplands            12  0.05   0.00
12              Urban and Built-up Lands            13  0.00   0.03
13  Cropland, Natural Vegetation, Mosaic            14  0.05   0.00
14                Permanent Snow and Ice            15  0.00   0.00
15                                Barren            16  1.00   1.00
16                          Water Bodies            17  0.00   0.00

In [ ]:
# Total number of iterations in the original loop
total_iterations = 360 * 360

# Number of iterations completed before timeout
completed_iterations = (64 + 180) * 360 + 129

# Calculate the percentage
percentage_complete = (completed_iterations / total_iterations) * 100

print(f"Percentage complete: {percentage_complete:.2f}%")

In [ ]:
# Given latitude and longitude values where the code stopped (final_lat and final_lon)
final_lat = 20  # Replace with the actual final latitude value
final_lon = -161  # Replace with the actual final longitude value

# Calculate the number of iterations completed based on the actual latitude and longitude values
min_lat = -85
max_lat = 86
min_lon = -180
max_lon = -150
total_latitude_steps = len(range(min_lat, max_lat))  # Total latitude steps
total_longitude_steps = len(range(min_lon, max_lon))  # Total longitude steps
iterations_completed = len(range(min_lat,final_lat)) * total_longitude_steps + len(range(min_lon,final_lon+1))

# Calculate the total number of iterations for the full range
total_iterations = total_latitude_steps * total_longitude_steps

# Calculate the percentage of completion
percentage_complete = (iterations_completed / total_iterations) * 100

print(f"Percentage of total iterations completed: {percentage_complete:.2f}%")


In [ ]:
# Combine results files
# Specify the directory containing your CSV files
results_directory = os.path.join(os.getcwd(),'results')


In [ ]:

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(results_directory) if file.endswith('.csv')]

# Initialize an empty list to store data frames
dataframes = []

# Read each CSV file and append its data frame to the list
for csv_file in csv_files:
    file_path = os.path.join(results_directory, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all data frames into a single data frame
combined_df = pd.concat(dataframes, ignore_index=True)
# Sort the combined DataFrame by "Latitude" and then "Longitude"
combined_df = combined_df.sort_values(by=["Latitude", "Longitude"])

# Reset the index after sorting
combined_df = combined_df.reset_index(drop=True)

In [ ]:

# Specify the path for the combined CSV file
combined_csv_path = os.path.join(results_directory, '2050_lcoa_global.csv')


In [ ]:

# Write the combined data frame to a new CSV file
combined_df.to_csv(combined_csv_path, index=False)

print(f'Combined data written to {combined_csv_path}')

In [ ]:
global_df = pd.read_csv(combined_csv_path)
global_df
# Need to calculate Electricity_cost_frac
# Should be Solar*Solar_cost/MW + Wind*cost/MW /(Solar*Solar_cost/MW + Wind*cost/MW + Electrolysis*cost/MW

In [3]:


data = plc.all_locations(data_dir)

selected = data.Solars[2].sel(
            dict(latitude=7, longitude=-61), method='nearest'
        ).Solar.values

Solars = data.Solars
Winds = data.Winds
Solars[0]


all_locations data_dir: /Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data
all_locations nc files: ['/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/Solar.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/Solar1.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/Solar2.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/SolarTracking.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/SolarTracking1.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/SolarTracking2.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/WindPowers.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/WindPowers1.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt/data/WindPowers2.nc', '/Users/carlopalazzi/Programming/shipping_sprint/lcoa_model/lcoa-opt

<xarray.Dataset>
Dimensions:    (time: 8760, latitude: 180, longitude: 120)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 ... 2019-12-31T23:00:00
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -180 -179 -178 -177 -176 ... -64 -63 -62 -61
Data variables:
    Solar      (time, latitude, longitude) float64 ...

In [4]:
Solars[1]

<xarray.Dataset>
Dimensions:    (time: 8760, latitude: 180, longitude: 120)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 ... 2019-12-31T23:00:00
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -60 -59 -58 -57 -56 -55 ... 54 55 56 57 58 59
Data variables:
    Solar      (time, latitude, longitude) float64 ...

In [5]:
Solars1 = xr.open_dataset(os.path.join(data_dir,'Solar1.nc'), engine="netcdf4")
Solars1

<xarray.Dataset>
Dimensions:    (time: 8760, latitude: 180, longitude: 120)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 ... 2019-12-31T23:00:00
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -60 -59 -58 -57 -56 -55 ... 54 55 56 57 58 59
Data variables:
    Solar      (time, latitude, longitude) float64 ...

In [10]:
# Ammonia production energy requirements
# Constants
desired_ammonia_production_tonnes = 10000000  # 10,000,000 tonnes
higher_heating_value_ammonia_MWh_per_tonne = 6.25  # MWh/t
duration_hours = 8760  # hours in a year

# Calculate the required energy in MWh
required_energy_MWh = desired_ammonia_production_tonnes * higher_heating_value_ammonia_MWh_per_tonne

# Calculate the required power in MW
required_power_MW = required_energy_MWh / duration_hours

print(f"Required power (MW): {required_power_MW}")


Required power (MW): 7134.703196347032
